In [2]:
import pandas as pd
import json
import requests
import time

In [1]:
path_dataset = "D:/datasets/flickr-30k-images"
path_captions = "D:/YandexDisk/datasets/flickr-30k-images-metadata.csv"

path_prepared_captions = "D:/flickr-30k-images-metadata.csv"

In [3]:
df_captions = pd.read_csv(path_captions, delimiter='|')
df_captions.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together


## Выбрать по миллиону символов

In [22]:
df_captions.iloc[-1]

image_name                                             998845445.jpg
 comment_number                                                    4
 comment           A man on a moored blue and white boat with hil...
Name: 153909, dtype: object

In [23]:
count_chars = 0
last_idx = -1


for idx in range(16460+1, 153909+1):
    count_chars += len(df_captions.iloc[idx, 2])
    if count_chars > 1000000:
        print(last_idx)
        break
    last_idx = idx

print(count_chars)

33234
1000103


In [27]:
count_chars = 0
last_idx = -1

for idx in range(16460+1, 33234+1):
    count_chars += len(df_captions.iloc[idx, 2])
    last_idx = idx

print(last_idx)
print(count_chars)

33234
999982


#### Первый миллион

In [18]:
df_captions_1 = df_captions[:16460]
df_captions_1.shape

(16460, 3)

In [19]:
df_captions_1.to_csv("D:/flickr-30k-images-metadata-1.csv", sep='|', encoding='utf-8', index=False)

#### Второй миллион

In [29]:
df_captions_2 = df_captions[16460+1:33234]
df_captions_2.shape

(16773, 3)

In [30]:
df_captions_2.to_csv("D:/flickr-30k-images-metadata-2.csv", sep='|', encoding='utf-8', index=False)

## 20 запросов в секунду

In [31]:
def fake_translate(text):
    return text[::-1]

In [32]:
fake_translate("Привет")

'тевирП'

In [108]:
df_for_fake = pd.read_csv("D:/YandexDisk/datasets/flickr-30k-images-metadata-for-fake.csv", delimiter='|')
df_for_fake.shape

(153910, 3)

In [110]:
df_for_fake.head(6)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together
5,10002456.jpg,0,Several men in hard hats are operating a giant...


In [124]:
def fake_requesting():
    start_idx = 0

    for idx in range(20, 16460+1, 20):
        start_time = time.time()

        df_for_twenty = df_for_fake[start_idx:idx]
        start_idx = idx

        for idx_for_twenty in df_for_twenty.index:
            df_for_twenty.at[idx_for_twenty, " comment"] = fake_translate(df_for_twenty.at[idx_for_twenty, " comment"])

        time_difference = time.time() - start_time

        if time_difference < 1.0:
            time.sleep(1.1 - time_difference)

In [113]:
%%time
fake_requesting()

Wall time: 15min 11s


## Перевод

Квоты — организационные ограничения, которые можно изменить по запросу в техническую поддержку.
Лимиты — технические ограничения, обусловленные особенностями архитектуры Yandex.Cloud. Изменение лимитов невозможно.

Yandex Translate. Квоты (4 апреля 2021):
Вызовов одного метода API в секунду – 20
Символов, отправленных на перевод, в час – 1 млн

In [134]:
IAM_TOKEN = "_____"
FOLDER_ID = "_____"

BEARER = "Bearer " + IAM_TOKEN
HEADERS = {
    "Content-type": "application/json",
    "Authorization": BEARER
}

def translate_with_yandex(text):
    data = {
        "folder_id": FOLDER_ID,
        "texts": text,
        "targetLanguageCode": "ru"
    }

    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate', headers=HEADERS, data=str(data))
    json_text = json.loads(response.text)

    return json_text["translations"][0]["text"]

In [135]:
translate_with_yandex("cloud")

'облако'

## Перевести первый миллион символов

In [132]:
df1 = pd.read_csv("D:/YandexDisk/datasets/flickr-30k-images-metadata-1.csv", delimiter='|')
df1.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together


In [133]:
def request_yandex_translate():
    start_idx = 0

    for idx in range(20, 16459+1, 20):
        start_time = time.time()

        df_for_twenty = df1[start_idx:idx]
        start_idx = idx

        for idx_for_twenty in df_for_twenty.index:
            df_for_twenty.at[idx_for_twenty, " comment"] = translate_with_yandex(df_for_twenty.at[idx_for_twenty, " comment"])

        time_difference = time.time() - start_time

        if time_difference < 1.0:
            time.sleep(1.01 - time_difference)

In [136]:
%%time
request_yandex_translate()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2019' in position 80: Body ('’') is not valid Latin-1. Use body.encode('utf-8') if you want to send it encoded in UTF-8.

In [140]:
df1.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,Два молодых парня с лохматыми волосами смотрят...
1,1000092795.jpg,1,Два молодых белых самца находятся снаружи возл...
2,1000092795.jpg,2,Двое мужчин в зеленых рубашках стоят во дворе
3,1000092795.jpg,3,"Мужчина в синей рубашке, стоящий в саду"
4,1000092795.jpg,4,"Два друга наслаждаются временем, проведенным в..."


In [139]:
df1.to_csv("D:/russian-flickr-30k-images-metadata-1.csv", sep='|', encoding='utf-8', index=False)